In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!pip install tensorflow

In [ ]:
#!pip install Keras

In [ ]:
def load_data(path):
    with np.load(path) as f:
        x_train, y_train = f['x_train'], f['y_train']
        x_test, y_test = f['x_test'], f['y_test']
        return (x_train, y_train), (x_test, y_test)
(x_train, y_train), (x_test, y_test) = load_data('../input/mnist-numpy/mnist.npz')

In [ ]:
import matplotlib.pyplot as plt
print(x_train.shape,x_test.shape)
for i in range(10):
    plt.figure(figsize=(5,5))
    plt.imshow(x_train[i])
    plt.title(y_train[i])
    plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import matplotlib.pyplot as plt
from keras.utils import to_categorical

Image data(3 dim) -reshape to -tensor shape(4 dim)

In [ ]:
#Reshape of img data to the tensor shape
img_ht,img_wd,channels=28,28,1 #1-greyscale image
x_train=x_train.reshape(x_train.shape[0],img_ht,img_wd,channels) #for tensor representatiom
x_test=x_test.reshape(x_test.shape[0],img_ht,img_wd,channels)
print(x_test.shape,x_test.shape)

Normalize the img pixels to 0 and 1 ->divided by max RGB code.

In [ ]:
x_train=x_train.astype('float32')/255
x_test=x_test.astype('float32')/255
print((x_train.shape),(x_test.shape))

One-hot encoding of label data

In [ ]:
num_classes=10
y_train1 =to_categorical(y_train, num_classes)
y_test1 = to_categorical(y_test, num_classes)
print(y_train1.shape,y_test1.shape)

Split the dataset as train and test

In [ ]:
from sklearn.model_selection import train_test_split
xtrain,xval,ytrain_1,yval_1=train_test_split(x_train,y_train1,test_size=0.16666,shuffle=True)
print(xtrain.shape)
print(ytrain_1.shape)


Sequential model selection

1)Convulation layers:
    Created three convulation layer.
    Parameters->no of filters,kernal/filter size,activation fn,padding
2)Pooling layer:
    after every convultion,pooling is done-reduce the parametres as well as Dimensionality reduction(retains the imp features/portion of the image).
3)Do 1 and 2 untill we get satified or more imp features.
4)Flattening of output.(vectors)
5)FCC
6)Activation-softmaxor sigmooid based on task

In [ ]:
def create_model():
    model = Sequential()
    
    model.add(Conv2D(32,(3,3),activation='relu',padding='same',input_shape=[img_ht,img_wd,channels])) #buiding
    #model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2)) #drop some neurons to avoid overfitting
    
    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    
    model.add(Dense(128,activation='relu')) #FCC
    model.add(Dropout(0.2))
    
    model.add(Dense(num_classes,activation='softmax'))
    model.compile(optimizer='adam',loss=tf.keras.losses.categorical_crossentropy,metrics=['accuracy'])
    return model
    
model=create_model()
model.summary()


fiiting tha train data nd evaluate with validation data.Do utill we get some good accuracy

In [ ]:
result=model.fit(xtrain,ytrain_1,epochs=2,batch_size=64, verbose=1,
          validation_data=(xval, yval_1))


In [ ]:
result.history


Evalution with our test data

In [ ]:
y_test.shape
test_acc,test_loss=model.evaluate(x_test,y_test1,batch_size=64)

Prediction

In [ ]:
pred=model.predict(x_test)

Checking our prediction data

In [ ]:
import numpy as np
print(np.argmax(pred[0]))
print(np.argmax(y_test1[0]))
print(x_test)

In [ ]:
ct=0
x_test=x_test.reshape(x_test.shape[0],img_ht,img_wd)
for i in range(10):
    plt.figure(figsize=(5,5))
    x=np.argmax(pred[i])
    y=np.argmax(y_test[i])
    if(x==y):
        ct+=1
    elif(x!=y):
        ct-=1    
    plt.imshow(x_test[i])
    plt.title(y_test[i])
    plt.show()
print(ct)